<a href="https://colab.research.google.com/github/Tharindu-dd/ML_Project/blob/main/Sri_Lanka_Vehicle_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>